In [386]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import math
from scipy.optimize import minimize
from scipy.spatial import ConvexHull
from scipy.linalg import norm
from scipy.optimize import minimize_scalar

path= 'C:\\Users\\matteo posenato\\Documents\\optimization\\project\\banana.csv'
data = pd.read_csv(path, sep=',')

In [387]:

data.shape

(5300, 3)

In [388]:
data.head()

,At1,At2,Class
0,1.140,-0.114,-1
1,-1.520,-1.150,1
2,-1.050,0.720,-1
3,-0.916,0.397,1
4,-1.090,0.437,1


In [389]:
def regularize_data(data):
    data = data[data.At1 < 1]
    data = data[data.At1 > -1]
    data = data[data.At2 < 1]
    data = data[data.At2 > -1]
    data = data.reset_index(drop = True)
    return data
def canonical_basis_vector(size, index):
    vector = np.zeros(size)
    vector[index] = 1
    return vector
def create_d_t(d, A, w):
    m = len(d)
    delta = np.sum(d * np.log(d)) + np.log(m)
    result = np.dot(d.T, np.dot(A, w))
    result += (1/m) * delta
    return result

In [390]:
data_r = regularize_data(data)

In [391]:
data_r.shape

(1745, 3)

In [392]:
data_r.head()

,At1,At2,Class
0,-0.916,0.3970,1
1,-0.584,0.0937,1
2,-0.482,-0.4850,1
3,-0.122,-0.8080,-1
4,-0.541,-0.3320,1


In [393]:
def weak_learner(data, weights=None):
    X = data.iloc[:, :-1]
    y = data.Class
    model = DecisionTreeClassifier(max_depth=2)
    if weights is not None:
        model.fit(X, y, sample_weight=weights)
    else:
        model.fit(X,y)
    h = model.predict(X)
    #return our hypothesis space
    return h, X, y
    

In [394]:
h, X, y = weak_learner(data_r)

In [395]:
def create_A(h, X, y):
    m = len(h)  
    #create A
    A = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            value = y[i] * h[j]
            A[i][j] = value
    return A, m



In [396]:
A, m = create_A(h, X, y)

In [397]:
d0 = np.zeros(m)
for i in range(m):
    d0[i] = 1/m 


In [398]:
hj_1, X, y = weak_learner(data_r, weights=d0)

In [408]:
e_j = canonical_basis_vector(m, 1)
w = np.zeros(m)
w[1] = e_j[1]

In [409]:
print(w)

[0. 1. 0. ... 0. 0. 0.]


In [373]:
result = minimize(create_d_t, x0=d0, args=(A, w[1]))


C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_8568\1915115368.py:14: RuntimeWarning: invalid value encountered in log
  delta = np.sum(d * np.log(d)) + np.log(m)
C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_8568\1915115368.py:14: RuntimeWarning: invalid value encountered in log
  delta = np.sum(d * np.log(d)) + np.log(m)
C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_8568\1915115368.py:14: RuntimeWarning: invalid value encountered in log
  delta = np.sum(d * np.log(d)) + np.log(m)


In [374]:
print(result.x[1])

-24.69321998470305


In [375]:
d0[1] = result.x[1]

In [376]:
e_j_next = canonical_basis_vector(m, 2)
h_j_nect = h[2]
E_t = []
E_t.insert(0, e_j)
E_t.insert(1, e_j_next)

[array([0., 1., 0., ..., 0., 0., 0.]), array([0., 0., 1., ..., 0., 0., 0.])]

In [377]:
lamb = []
value_1 = np.dot(d0[1].T, np.dot(A, (e_j_next-w[1])))
value_2 = np.dot(A, e_j_next - w[1])
value_3 = norm(value_2, ord=np.inf)
value = value_1 / value_3
lamb.insert(0, np.clip(value, 0, 1))

C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_8568\3999495292.py:5: RuntimeWarning: invalid value encountered in divide
  value = value_1 / value_3


In [385]:
print(value)

[nan nan nan ... nan nan nan]


In [379]:
print(w[1])

[0. 1. 0. ... 0. 0. 0.]


In [380]:
w_1= []
r = (e_j_next - w[1])
r_1 = (lamb[0]*r)
w_1.insert(0, w[1] + r_1 )

In [381]:
print(w_1)

[array([nan, nan, nan, ..., nan, nan, nan])]


In [279]:
w_2 = []
my_f = lambda x, A, w: np.dot(d0.T, np.dot(A, x))
result = minimize(my_f, x0=d0, args=(A, w))
w_2.append(result.x)

In [302]:
w_f[1]

array([ 6498909.05520857,  6498884.36141552,  6498909.05520857, ...,
       -6502316.6136726 , -6502316.6136726 ,  6498909.05520857])

In [311]:
w_f = np.array([w_1[0], w_2[0]])
def f(w):
    return np.dot(d0.T, -np.dot(A, w))
def f_support(x, w):
    if x == 0:
        return f(w[0])
    elif x == 1:
        return f(w[1])

# Trova l'argomento che minimizza la funzione per il primo vettore
result_1 = minimize_scalar(f_support, args=(w_f,), bounds=(0, 0), method='bounded')
arg_min_1 = result_1.x

# Trova l'argomento che minimizza la funzione per il secondo vettore
result_2 = minimize_scalar(f_support, args=(w_f,), bounds=(1, 1), method='bounded')
arg_min_2 = result_2.x
# Confronta i risultati per determinare quale valore minimizza di più la funzione
if arg_min_1 < arg_min_2:
    # Il primo vettore minimizza di più la funzione
    print("Il primo vettore minimizza di più la funzione.")
    print(arg_min_1)
    w[2] = arg_min_1
elif arg_min_1 > arg_min_2:
    # Il secondo vettore minimizza di più la funzione
    print("Il secondo vettore minimizza di più la funzione.")
    w[2] = arg_min_2
else:
    # Entrambi i valori sono uguali
    print("Entrambi i valori minimizzano la funzione allo stesso modo.")

Il primo vettore minimizza di più la funzione.
0.0


In [411]:
print(w)

[0. 1. 0. ... 0. 0. 0.]


In [415]:
#seconda parte all'interno del ciclo for
T = 5
E_t = []
E_t.insert(0, e_j)
lamb = []
lamb.insert(0, np.zeros(m))
for t in range(1, T):
    #create the distribution d_t
    t_n = t + 1
    result = minimize(create_d_t, x0=d0, args=(A, w))
    d0[t] = result.x[1]
    #obtain the hypothesis h_j+1 and e_j+1
    e_j_next = canonical_basis_vector(m, t_n)
    print(e_j_next)
    h_j_nect = h[t_n]
    E_t.insert(t_n, e_j_next)
    #compute the FW weight
    value_1 = np.dot(d0[t].T, np.dot(A, (e_j_next-w[t])))
    value_2 = np.dot(A, e_j_next - w[t])
    value_3 = norm(value_2, ord=np.inf)
    value = value_1 / value_3
    lamb.insert(t, np.clip(value, 0, 1))
    w_1= []
    r = (e_j_next - w[t])
    w_1.insert(0, w[t] + (lamb[t]*r))
    #compute the secondary weight
    w_2 = []
    my_f = lambda x, A, w: np.dot(d0.T, np.dot(A, x))
    result = minimize(my_f, x0=d0, args=(A, w))
    w_2.append(result.x)
    #update the weight
    w_f = np.array([w_1[0], w_2[0]])
    def f(w):
        return np.dot(d0.T, -np.dot(A, w))
    def f_support(x, w):
        if x == 0:
            return f(w[0])
        elif x == 1:
            return f(w[1])

    # Trova l'argomento che minimizza la funzione per il primo vettore
    result_1 = minimize_scalar(f_support, args=(w_f,), bounds=(0, 0), method='bounded')
    arg_min_1 = result_1.x

    # Trova l'argomento che minimizza la funzione per il secondo vettore
    result_2 = minimize_scalar(f_support, args=(w_f,), bounds=(1, 1), method='bounded')
    arg_min_2 = result_2.x
    # Confronta i risultati per determinare quale valore minimizza di più la funzione
    if arg_min_1 < arg_min_2:
        # Il primo vettore minimizza di più la funzione
        print("Il primo vettore minimizza di più la funzione.")
        print("argmin1: {}".format(arg_min_1))
        w[t_n] = arg_min_1
    elif arg_min_1 > arg_min_2:
        # Il secondo vettore minimizza di più la funzione
        print("Il secondo vettore minimizza di più la funzione.")
        print("argmin2: {}".format(arg_min_2))
        w[t_n] = arg_min_2
    else:
        # Entrambi i valori sono uguali
        print("Entrambi i valori minimizzano la funzione allo stesso modo.")

C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_8568\1915115368.py:14: RuntimeWarning: invalid value encountered in log
  delta = np.sum(d * np.log(d)) + np.log(m)


[0. 0. 1. ... 0. 0. 0.]
Il primo vettore minimizza di più la funzione.
argmin1: 0.0
[0. 0. 0. ... 0. 0. 0.]
Il primo vettore minimizza di più la funzione.
argmin1: 0.0
[0. 0. 0. ... 0. 0. 0.]
Il primo vettore minimizza di più la funzione.
argmin1: 0.0
[0. 0. 0. ... 0. 0. 0.]
Il primo vettore minimizza di più la funzione.
argmin1: 0.0


In [ ]:
#pairwise
my_f = lambda x, A, e_j: np.dot(d0.T, np.dot(A, x))
e_away = minimize(my_f, x0=e_j, args=(A, d0))
def compute_alpha(w, e):
    alpha = w[np.nonzero(e)][0]
for e in E_t:
    alpha[e] = compute_alpha(w, e)
lamb_max = alpha[e_away]
lamb = min([i for i in range(0, lambda_max + 1)], key=lambda i: f(-A(wt + lambda * (ejt_1 - e_away))))
w_1 = w + lamb* (e_j_next - e_away)

In [ ]:
def FW_ss_LPBoost(A, m, h, T):
    #create d_0
    d = np.zeros(m)
    for i in range(m):
        d[i] = 1/m 
    d_t = []
    #create vector basis
    e_j = canonical_basis_vector(m, 1)
    w = np.zeros(m)
    w[1] = e_j[1]
    w_1 = np.zeros(T)
    w_2 = np.zeros(T)
    epsilon = 0.01
    lamb = np.zeros(T)
    #for cicle
    for t in range(1, T):
        #compute the distibution dt
        result = minimize(create_d_t, x0=d, args=(A, w))
        d_t.append(result.x)
        e_j_next = canonical_basis_vector(m, t+1)
        h_j_nect = h[t+1]
        E_t_next = canonical_basis_vector(t+1, t)
        #create lambda_t 
        value_1 = np.dot(d_t[t-1].T, np.dot(A, (e_j_next-w[t])))
        
        lamb[t] = np.clip(value, 0, 1)
        #CH = ConvexHull(E_t_next)
        #vertices = E_t_next[CH.vertices]
        #compute the FW weights
        r = (e_j_next - w[t])
        w_1[t-1] = w[t] + (lamb[t]*r)
        w_d = np.argmax(w)
        w_2[t-1] = minimize(np.dot(d.T, np.dot(A, w_d)), x0=d)
        w_f = [w_1[t+1], w_2[t +1]]
        w[t+1] = minimize (np.dot(d.T, np.dot(A, w_f)), x0=w_f)
    return d, w, d_t

In [ ]:
def FW_ss_LPBoost(A, m, h, T):
    #create d_0
    d = np.zeros(m)
    for i in range(m):
        d[i] = 1/m 
    d_t = []
    #create vector basis
    e_j = canonical_basis_vector(m, 1)
    w = np.zeros(m)
    w[1] = e_j[1]
    w_1 = np.zeros(T)
    w_2 = np.zeros(T)
    epsilon = 0.01
    lamb = np.zeros(T)
    #for cicle
    for t in range(1, T):
        #compute the distibution dt
        result = minimize(create_d_t, x0=d, args=(A, w))
        d_t.append(result.x)
        e_j_next = canonical_basis_vector(m, t+1)
        h_j_nect = h[t+1]
        E_t_next = canonical_basis_vector(t+1, t)
        #create lambda_t 
        value = np.dot(d_t[t-1].T, np.dot(A, (e_j_next-w[t])))
        lamb[t] = np.clip(value, 0, 1)
        #CH = ConvexHull(E_t_next)
        #vertices = E_t_next[CH.vertices]
        #compute the FW weights
        r = (e_j_next - w[t])
        w_1[t-1] = w[t] + (lamb[t]*r)
        w_d = np.argmax(w)
        w_2[t-1] = minimize(np.dot(d.T, np.dot(A, w_d)), x0=d)
        w_f = [w_1[t+1], w_2[t +1]]
        w[t+1] = minimize (np.dot(d.T, np.dot(A, w_f)), x0=w_f)
    return d, w, d_t

In [116]:
def FW_ss_LPBoost_1(A, m, h, T):
    #create d_0
    d = np.full(m, 1/m)
    d_t = []
    #create vector basis
    e_j = canonical_basis_vector(m, 1)
    w = np.zeros(m)
    w[1] = e_j[1]
    w_1 = np.zeros(T + 1)
    w_2 = np.zeros(T + 1)
    epsilon = 0.01
    lamb = np.zeros(T + 1)
    #for cycle
    for t in range(1, T):
        #compute the distribution dt
        result = minimize(create_d_t, x0=d, args=(A, w))
        d_t.append(result.x)
        e_j_next = canonical_basis_vector(m, t+1)
        h_j_next = h[t+1]
        E_t_next = canonical_basis_vector(t+1, t)
        #create lambda_t 
        value = np.dot(d_t[t-1].T, np.dot(A, (e_j_next-w[t])))
        lamb[t] = np.clip(value, 0, 1)
        #CH = ConvexHull(E_t_next)
        #vertices = E_t_next[CH.vertices]
        #compute the FW weights
        r = (e_j_next - w[t])
        r_1 = w[t] + (lamb[t]*r)
        print(r_1)
        w_1[t-1] = r_1
        w_d = np.argmax(w)
        w_2[t-1] = minimize(lambda x: np.dot(d.T, np.dot(A, x)), x0=d).x
        w_f = [w_1[t-1], w_2[t-1]]
        w[t] = minimize(lambda x: np.dot(d.T, np.dot(A, x)), x0=w_f).x
    return d, w, d_t

In [106]:
A, m = create_A(data_r, h)

In [119]:
print(A, len(A))

[[-0.363652  -0.363652  -0.363652  ... -0.363652   0.363652  -0.363652 ]
 [-0.0547208 -0.0547208 -0.0547208 ... -0.0547208  0.0547208 -0.0547208]
 [ 0.23377    0.23377    0.23377   ...  0.23377   -0.23377    0.23377  ]
 ...
 [-0.637824  -0.637824  -0.637824  ... -0.637824   0.637824  -0.637824 ]
 [ 0.593668   0.593668   0.593668  ...  0.593668  -0.593668   0.593668 ]
 [ 0.03621    0.03621    0.03621   ...  0.03621   -0.03621    0.03621  ]] 1745


In [117]:
d, w, d_t = FW_ss_LPBoost_1(A, m, h, T=2)

C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_8568\1915115368.py:14: RuntimeWarning: invalid value encountered in log
  delta = np.sum(d * np.log(d)) + np.log(m)
C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_8568\1915115368.py:14: RuntimeWarning: invalid value encountered in log
  delta = np.sum(d * np.log(d)) + np.log(m)
C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_8568\1915115368.py:14: RuntimeWarning: invalid value encountered in log
  delta = np.sum(d * np.log(d)) + np.log(m)


[0. 0. 1. ... 0. 0. 0.]


ValueError: setting an array element with a sequence.

In [ ]:
print(d_t)